In [6]:
import os
import sys
import json
import datetime
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn import visualize

# Path to trained weights file
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# DEFAULT_IMAGE_DIR = os.path.join(ROOT_DIR,'samples/flood_river/test_img')
DEFAULT_IMAGE_DIR = os.path.join(ROOT_DIR,'samples/wheel_level/test_img')

In [2]:
class TireConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "wheel_level"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [3]:
def color_splash(image, mask):
    """Apply color splash effect.
    image: RGB image [height, width, 3]
    mask: instance segmentation mask [height, width, instance count]

    Returns result image.
    """
    # Make a grayscale copy of the image. The grayscale copy still
    # has 3 RGB channels, though.
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    
    gray = np.zeros(image.shape)
    # Copy color pixels from the original color image where mask is set
    if mask.shape[-1] > 0:
        # We're treating all instances as one, so collapse the mask into one layer
        mask = (np.sum(mask, -1, keepdims=True) >= 1)
        splash = np.where(mask, image, gray).astype(np.uint8)
    else:
        splash = gray.astype(np.uint8)
    return splash

def detect_and_color_splash(model, image_name=None, video_path=None):
    assert image_name or video_path
    image_path = os.path.join(DEFAULT_IMAGE_DIR, image_name)
    
    flood_depth_pythagoras  = []
    flood_depth_hough  = []
    # Image or video?
    if image_path:
        # Run model detection and generate the color splash effect
        print("Running on {}".format(image_path))
        # Read image
        image = cv2.imread(image_path)

        # Detect objects
        r = model.detect([image], verbose=1)[0]
        #J:check
#         print('r', r)
#         print('r[class_ids]', r['class_ids'])
#         print('r[rois]', r['rois'])


        # Color splash
        splash = color_splash(image, r['masks'])
        # Save output
        file_name = image_name
        cv2.imwrite('ans/'+file_name+'.jpg', splash)
          
#         imgray = cv2.cvtColor(splash.astype(np.uint8), cv2.COLOR_BGR2GRAY)     
#         ret, thresh = cv2.threshold(imgray, 0, 255, 0)
#         circles = cv2.HoughCircles(thresh,cv2.HOUGH_GRADIENT,1,100,param1=200,param2=10,minRadius=10,maxRadius=500)
#         cv2.imwrite('mask.jpg',thresh)    

       
    elif video_path:
        # Video capture
        vcapture = cv2.VideoCapture(video_path)
        width = int(vcapture.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(vcapture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = vcapture.get(cv2.CAP_PROP_FPS)

        # Define codec and create video writer
        file_name = "splash_{:%Y%m%dT%H%M%S}.avi".format(datetime.datetime.now())
        vwriter = cv2.VideoWriter(file_name,
                                  cv2.VideoWriter_fourcc(*'MJPG'),
                                  fps, (width, height))

        count = 0
        success = True
        while success:
            print("frame: ", count)
            # Read next image
            success, image = vcapture.read()
            if success:
                # OpenCV returns images as BGR, convert to RGB
                image = image[..., ::-1]
                # Detect objects
                r = model.detect([image], verbose=0)[0]
                # Color splash
                splash = color_splash(image, r['masks'])
                # RGB -> BGR to save image to video
                splash = splash[..., ::-1]
                # Add image to video writer
                vwriter.write(splash)
                count += 1
        vwriter.release()
    print("Saved to ", file_name)
    
    return r



# note 2020-07-28

/home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/logs/lood_river20200728T1333

GPU_COUNT = 8
IMAGES_PER_GPU = 1
STEPS_PER_EPOCH = 100
epochs=30
real    61m9.886s
user    435m52.298s
sys     267m7.060s 

# note 2023-04-23

retest wheel water level!

In [4]:
if __name__ == '__main__':
    
#     tire_weights_path = '/home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/logs/flood_river20200728T1333/mask_rcnn_flood_river_0030.h5'
    tire_weights_path = '/home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/logs/wheel_level20210423T1444/mask_rcnn_wheel_level_0003.h5'
  
    class InferenceConfig(TireConfig):
        # Set batch size to 1 since we'll be running inference on
        # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
    tire_config = InferenceConfig()
    #tire_config.display()
    
    # Create model
    tire_model = modellib.MaskRCNN(mode="inference", config=tire_config, model_dir='')
       
    # Load weights
    tire_model.load_weights(tire_weights_path, by_name=True)   





Re-starting from epoch 3


In [7]:
#image_path = '1400720642688.jpg'

# Train or evaluate
for root, dirs, files in os.walk(DEFAULT_IMAGE_DIR):
    for f in files:
        print('Image name =', f)
        tire_roi = detect_and_color_splash(tire_model, image_name=f)        
        image_path1 = os.path.join(DEFAULT_IMAGE_DIR, f)    
        img = cv2.imread(image_path1)
        print('class_ids =', tire_roi['class_ids'])
        masks = tire_roi['masks']
        # Copy color pixels from the original color image where mask is set
        gray = cv2.applyColorMap(img, 7)

        alpha = 0.5
        

        for i in range(masks.shape[-1]):     
            if tire_roi['class_ids'][i] == 1:   
                color = [0,255,0]
                for c in range(3):
                    img[:, :, c] = np.where(masks[:,:,i] == 1, img[:, :, c] * (1 - alpha) + alpha * color[c], img[:, :, c])
            if tire_roi['class_ids'][i] == 2:     
                color = [230,255,102]
                for c in range(3):
                    img[:, :, c] = np.where(masks[:,:,i] == 1, img[:, :, c] * (1 - alpha) + alpha * color[c], img[:, :, c])
            if tire_roi['class_ids'][i] == 3:     
                color = [0,0,255]
                for c in range(3):
                    img[:, :, c] = np.where(masks[:,:,i] == 1, img[:, :, c] * (1 - alpha) + alpha * color[c], img[:, :, c])


        cv2.imwrite('test_result/'+f,img)
        


Image name = 24.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/24.jpg
Processing 1 images
image                    shape: (720, 1280, 3)        min:    7.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Saved to  24.jpg
class_ids = [1]
Image name = flood_training_75.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/flood_training_75.jpg
Processing 1 images
image                    shape: (88, 185, 3)          min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors

Saved to  flood_training_7.jpg
class_ids = [1 1]
Image name = flood_training_51.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/flood_training_51.jpg
Processing 1 images
image                    shape: (406, 895, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Saved to  flood_training_51.jpg
class_ids = []
Image name = 6.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/6.jpg
Processing 1 images
image                    shape: (720, 1280, 3)        min:   20.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)   

Saved to  flood_training_80.jpg
class_ids = []
Image name = 2-1.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/2-1.jpg
Processing 1 images
image                    shape: (720, 1280, 3)        min:    1.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Saved to  2-1.jpg
class_ids = [1 1]
Image name = 6-2.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/6-2.jpg
Processing 1 images
image                    shape: (768, 1024, 3)        min:   13.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 102

Saved to  flood_training_17.jpg
class_ids = []
Image name = flood_training_89.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/flood_training_89.jpg
Processing 1 images
image                    shape: (448, 996, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Saved to  flood_training_89.jpg
class_ids = []
Image name = flood_training_91.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/flood_training_91.jpg
Processing 1 images
image                    shape: (519, 1139, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas 

Saved to  18.jpg
class_ids = [1 1 1]
Image name = flood_training_44.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/flood_training_44.jpg
Processing 1 images
image                    shape: (36, 83, 3)           min:    0.00000  max:  246.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  141.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Saved to  flood_training_44.jpg
class_ids = []
Image name = flood_training_57.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/flood_training_57.jpg
Processing 1 images
image                    shape: (388, 840, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas           

Saved to  flood_training_85.jpg
class_ids = []
Image name = 3-1.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/3-1.jpg
Processing 1 images
image                    shape: (768, 1024, 3)        min:   13.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Saved to  3-1.jpg
class_ids = [1]
Image name = flood_training_36.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/flood_training_36.jpg
Processing 1 images
image                    shape: (313, 931, 3)         min:   22.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)              

Saved to  flood_training_90.jpg
class_ids = [1]
Image name = flood_training_63.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/flood_training_63.jpg
Processing 1 images
image                    shape: (441, 917, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Saved to  flood_training_63.jpg
class_ids = []
Image name = flood_training_55.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/flood_training_55.jpg
Processing 1 images
image                    shape: (401, 1000, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  148.10000  float64
image_metas

Saved to  5-2.jpg
class_ids = [1 1]
Image name = 9.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/9.jpg
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Saved to  9.jpg
class_ids = [1]
Image name = flood_training_32.jpg
Running on /home/u3148947/twcc_gpfs/MaskRCNN/Mask_RCNN/samples/wheel_level/test_img/flood_training_32.jpg
Processing 1 images
image                    shape: (176, 392, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000 